Copyright 2024 Google, LLC. This software is provided as-is,
without warranty or representation for any use or purpose. Your
use of it is subject to your agreement with Google.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# How to use Batch Predicitons with Gemini

This notebook outlines how to interact with Vertex AI's Gemini models to call external API's using Function Calling. More info can be found at https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/batch-prediction-gemini

## Prepare the python development environment

First, let's identify any project specific variables to customize this notebook to your GCP environment. Change YOUR_PROJECT_ID with your own GCP project ID.

In [ ]:
project_id = "YOUR_PROJECT_ID"
location = "global"
region = "us-central1"
bq_dataset_id = "gemini_batch_test"
bq_table = "batch_input_table"
model_ver = "gemini-1.0-pro-002"

Install any needed python modules from our requirements.txt file. Most Vertex Workbench environments include all the packages we'll be using, but if you are using an external Jupyter Notebook or require any additional packages for your own needs, you can simply add them to the included requirements.txt file an run the folloiwng commands.

In [ ]:
#pip install -r requirements.txt

Now we will import all required modules. For our purpose, we will be utilizing the following:

- vertexai - Them primary library for working with the Vertex AI Platform on GCP 
- BatchPredictionJob - Used to submit and manage batch prediction jobs with Gemini

In [ ]:
import time
import vertexai
from vertexai.preview.batch_prediction import BatchPredictionJob

## Define and submit a Batch Prediction job for Gemini

Initialize vertexai

In [ ]:
vertexai.init(project=project_id, location=region)

Next we'll create the Gemini batch prediction job

In [ ]:
job = BatchPredictionJob.submit(
    model_ver,   # source_model 
    #"gs://rkiles-test/gemini-batch/batch_data2.json", # input URI if using GCS
    input_dataset = f'bq://{project_id}.{bq_dataset_id}.{bq_table}',  # input dataset if using BQ
    output_uri_prefix = f'bq://{project_id}.{bq_dataset_id}'  # This will generate a new output table in BQ
)

View and monitor the job status. You can also view the status in the GCP Cloud Console under Vertex AI -> Batch Predictions

In [ ]:
# Check job status
print(f"Job resouce name: {job.resource_name}")
print(f"Model resource name with the job: {job.model_name}")
print(f"Job state: {job.state.name}")

# Refresh the job until complete
while not job.has_ended:
  time.sleep(5)
  job.refresh()

# Check if the job succeeds
if job.has_succeeded:
  print("Job succeeded!")
else:
  print(f"Job failed: {job.error}")

Check the location of the output

In [ ]:
print(f"Job output location: {job.output_location}")

List all the GenAI batch prediction jobs under the project

In [ ]:
for bpj in BatchPredictionJob.list():
  print(f"Job ID: '{bpj.name}', Job state: {bpj.state.name}, Job model: {bpj.model_name}")